In [13]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


PageRank with Eigen Decompositions

Additional SVD Application
An interesting use of SVD that I recently came across was as a step in the de-biasing of Word2Vec word embeddings,Word2Vec is a useful library released by Google that represents words as vectors. The similarity of vectors captures semantic meaning, and analogies 

Relationship between SVD and Eigen Decomposition: the left-singular vectors of A are the eigenvectors of AAT. The right-singular vectors of A are the eigenvectors of ATA. The non-zero singular values of A are the square roots of the eigenvalues of ATA (and AAT).

SVD is a generalization of eigen decomposition. Not all matrices have eigen values, but ALL matrices have singular values.

In [10]:
import os, numpy as np, pickle
from bz2 import BZ2File
from datetime import datetime
from pprint import pprint
from time import time
from tqdm import tqdm_notebook
from scipy import sparse

from sklearn.decomposition import randomized_svd
#from sklearn.externals.joblib import Memory
from urllib.request import urlopen

In [15]:
PATH = '/content/drive/My Drive/'
#URL_BASE = 'http://downloads.dbpedia.org/3.5.1/en/'
filenames = ["redirects_en.nt.bz2", "page_links_en.nt.bz2"]

for filename in filenames:
  open(PATH+filename, 'wb')

In [16]:
redirects_filename = PATH+filenames[0]
page_links_filename = PATH+filenames[1]

In [17]:
DBPEDIA_RESOURCE_PREFIX_LEN = len("http://dbpedia.org/resource/")
SLICE = slice(DBPEDIA_RESOURCE_PREFIX_LEN + 1, -1)
def get_lines(filename): return (line.split() for line in BZ2File(filename))

In [18]:
#Loop through redirections and create dictionary of source to final destination

def get_redirect(targ, redirects):
    seen = set()
    while True:
        transitive_targ = targ
        targ = redirects.get(targ)
        if targ is None or targ in seen: break
        seen.add(targ)
    return transitive_targ
def get_redirects(redirects_filename):
    redirects={}
    lines = get_lines(redirects_filename)
    return {src[SLICE]:get_redirect(targ[SLICE], redirects) 
                for src,_,targ,_ in tqdm_notebook(lines, leave=False)}
redirects = get_redirects(redirects_filename)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:15: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  from ipykernel import kernelapp as app


EOFError: ignored

In [19]:
def add_item(lst, redirects, index_map, item):
    k = item[SLICE]
    lst.append(index_map.setdefault(redirects.get(k, k), len(index_map)))

limit=119077682 #5000000
# Computing the integer index map

index_map = dict() # links->IDs
lines = get_lines(page_links_filename)
source, destination, data = [],[],[]

for l, split in tqdm_notebook(enumerate(lines), total=limit):
    if l >= limit: break
    add_item(source, redirects, index_map, split[0])
    add_item(destination, redirects, index_map, split[2])
    data.append(1)


n=len(data)
n

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


EOFError: ignored

In [20]:
#The below steps are just to illustrate what info is in our data and how it is structured. They are not efficient.

index_map.popitem()

KeyError: ignored

In [ ]:
[i for i,x in enumerate(source) if x == 9991173]

In [ ]:
source[119077649], destination[119077649]

In [ ]:
for page_name, index in index_map.items():
    if index == 9991050:
        print(page_name)

In [ ]:
test_inds = [i for i,x in enumerate(source) if x == 9991050]
len(test_inds)

In [ ]:
test_inds[:5]

In [ ]:
test_dests = [destination[i] for i in test_inds]

In [ ]:
for page_name, index in index_map.items():
    if index in test_dests:
        print(page_name)

In [ ]:
#Create Matrix
#Below we create a sparse matrix using Scipy's COO format, and that convert it to CSR.

In [ ]:
X = sparse.coo_matrix((data, (destination,source)), shape=(n,n), dtype=np.float32)
X = X.tocsr()
del(data,destination, source)
X

In [ ]:
names = {i: name for name, i in index_map.items()}
mem_usage()

In [ ]:
#Save matrix so we don't have to recompute

pickle.dump(X, open(PATH+'X.pkl', 'wb'))
pickle.dump(index_map, open(PATH+'index_map.pkl', 'wb'))

X = pickle.load(open(PATH+'X.pkl', 'rb'))

index_map = pickle.load(open(PATH+'index_map.pkl', 'rb'))
names = {i: name for name, i in index_map.items()}

X

In [ ]:
#Power method
def show_ex(v):
    print(', '.join(names[i].decode() for i in np.abs(v.squeeze()).argsort()[-1:-10:-1]))
?np.squeeze

In [ ]:
S = sparse.csr_matrix(np.array([[1,2],[3,4]]))
Sr = S.sum(axis=0).A1; Sr

In [ ]:
S.indices
S.data
S.data / np.take(Sr, S.indices)

In [ ]:
def power_method(A, max_iter=100):
    n = A.shape[1]
    A = np.copy(A)
    A.data /= np.take(A.sum(axis=0).A1, A.indices)

    scores = np.ones(n, dtype=np.float32) * np.sqrt(A.sum()/(n*n)) # initial guess
    for i in range(max_iter):
        scores = A @ scores
        nrm = np.linalg.norm(scores)
        scores /= nrm
        print(nrm)

    return scores

In [ ]:
scores = power_method(X, max_iter=10)

In [ ]:
show_ex(scores)

In [3]:
#Compare to SVD

%time U, s, V = randomized_svd(X, 3, n_iter=3)

In [ ]:
# Top wikipedia pages according to principal singular vectors
show_ex(U.T[0])

In [ ]:
show_ex(U.T[1])

In [ ]:
show_ex(V[0])

In [ ]:
show_ex(V[1])

QR Algorithm

We used the power method to find the eigenvector corresponding to the largest eigenvalue of our matrix of Wikipedia links. This eigenvector gave us the relative importance of each Wikipedia page (like a simplified PageRank).*italicized text*

In [ ]:
#Pure QR
n = 6
A = np.random.rand(n,n)

AT = A @ A.T

def pure_qr(A, max_iter=50000):
    Ak = np.copy(A)
    n = A.shape[0]
    QQ = np.eye(n)
    for k in range(max_iter):
        Q, R = np.linalg.qr(Ak)
        Ak = R @ Q
        QQ = QQ @ Q
        if k % 100 == 0:
            print(Ak)
            print("\n")
    return Ak, QQ

In [ ]:
Ak, Q = pure_qr(A)

In [ ]:
#Let's compare to the eigenvalues:
np.linalg.eigvals(A)

In [ ]:
#Check that Q is orthogonal:

np.allclose(np.eye(n), Q @ Q.T), np.allclose(np.eye(n), Q.T @ Q)

In [ ]:
#Practical QR (QR with shifts)
#Add shifts to the QR algorithm
#Exercise: def practical_qr(A, iters=10):
#Exercise:     return Ak, Q
Ak, Q = practical_qr(A, 10)